In [4]:
import requests
from bs4 import BeautifulSoup

import datetime as dt
from dateutil import rrule

import pandas as pd
import json
import os

In [19]:
import time
import calendar


In [8]:
f_path = os.path.join('D:\python_project\sesac02\data', 'news_naver_IT_sample.csv')
df = pd.read_csv(f_path)
df.head(5)

,title,publisher,article_url,cmt_url,category2,publication_date,category1
0,"""애플, 2024년에도 챗GPT 앱 못 만든다""",지디넷코리아,https://n.news.naver.com/mnews/article/092/000...,https://n.news.naver.com/mnews/article/comment...,모바일,2023-08-02,IT
1,"엑스와이지, AI 로봇기술 해외진출 토대 마련",지디넷코리아,https://n.news.naver.com/mnews/article/092/000...,https://n.news.naver.com/mnews/article/comment...,모바일,2023-08-02,IT
2,뉴진스 공연 중 아이폰 '찰칵' vs BTS는 '온리 갤럭시'…한지붕 식구끼리 왜이래?,뉴시스,https://n.news.naver.com/mnews/article/003/001...,https://n.news.naver.com/mnews/article/comment...,모바일,2023-08-02,IT
3,카톡 '조용히 나가기'도 부담스럽다면…'조용한 채팅방' 기능 추가,한국경제,https://n.news.naver.com/mnews/article/015/000...,https://n.news.naver.com/mnews/article/comment...,모바일,2023-08-02,IT
4,"中, 미성년자 스마트기기 사용 '하루 2시간 이하'로 제한 방침",연합뉴스,https://n.news.naver.com/mnews/article/001/001...,https://n.news.naver.com/mnews/article/comment...,모바일,2023-08-02,IT


댓글 가져오기

In [13]:
cmt_url = df.iloc[0, 3]
cmt_url

'https://n.news.naver.com/mnews/article/comment/092/0002300844'

In [9]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'}

In [12]:
cmt_url = "https://news.naver.com/mnews/article/comment/011/0004386530"

In [14]:
res = requests.get(cmt_url, headers= headers)
res

<Response [200]>

In [15]:
res.text

'<!doctype html>\n<html lang="ko" data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36">\n\t<head>\n\t\t<meta charset="utf-8">\n\t\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" />\n\t\t<meta property="og:title" content="&quot;애플, 2024년에도 챗GPT 앱 못 만든다&quot;">\n\t\t<meta property="og:type" content="article">\n\t\t<meta property="og:url" content="https://n.news.naver.com/mnews/article/092/0002300844">\n\t\t<meta property="og:image" content="https://imgnews.pstatic.net/image/092/2023/08/02/0002300844_001_20230802233201189.jpg?type&#x3D;w800">\n\t\t<meta property="og:description" content="애플이 2024년까지는 챗GPT 앱을 만들 수 없을 것이라고 애플인사이더가 2일 애플 전문가인 궈밍치를 인용 보도했다. 보도에 따르면 궈밍치는 애플이 생성 인공지능(AI) 부문에선 경쟁사보다 다소 뒤진 상태라고 진단">\n\t\t<meta property="og:article:author" content="지디넷코리아 | 네이버">\n\t\t<m

In [16]:
soup = BeautifulSoup(res.text, 'html.parser')

In [17]:
matching_scripts = soup.select('script[src*="static.cbox"]')
matching_scripts

[]

API 요청 탐색

처음 로드된 페이지

In [90]:
rq_url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json"
cmt_url = 'https://n.news.naver.com/mnews/article/comment/092/0002300844'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
           "Referer": cmt_url}
p_time = calendar.timegm(dt.datetime.now().timetuple())

params = {
    "ticket": "news",
    "templateId": "default_it",
    "pool": "cbox5",
    # "_cv": "20240903144034",
    # "_callback": "jQuery33101711803272944905_1725349376637",
    "lang": "ko",
    "country": "KR",
    "objectId": "news092,0002300844",
    "categoryId": "",
    "pageSize": 20,
    "indexSize": 10,
    "groupId": "",
    "listType": "OBJECT",
    "pageType": "more",
    "page": 2,
    "currentPage": 1,
    "refresh": False,
    "sort": "FAVORITE",
    "current": 799229404260597805,
    "prev": 799229205719023626,
    "moreParam.direction": "next",
    "moreParam.prev": "100000m00000o062lj3we6iz2i",
    "moreParam.next": "0zik0zh000004062lj6flp1y5p",
    "includeAllStatus": True,
    "_": p_time
}



response = requests.get(rq_url, headers=headers, params=params)
soup = BeautifulSoup(response.text, 'html.parser')
res_text = response.text

x = res_text.split('(', 1)
js = json.loads(x[1][:-2])
c_list = js['result']['commentList']
len(c_list), js['result']['morePage']

(4,
 {'prev': '100000m00000o062lj3we6iz2i',
  'next': '0zik0zc000008062lfxurwlpi4',
  'start': '100000m00000o062lj3we6iz2i',
  'end': '0zik0zc000008062lfxurwlpi4'})

In [86]:
dict(js['result']).keys()

dict_keys(['commentList', 'pageModel', 'morePage', 'exposureConfig', 'count', 'sort', 'bestList'])

In [88]:
rq_url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json"
cmt_url = 'https://n.news.naver.com/mnews/article/comment/092/0002300844'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
           "Referer": cmt_url}
p_time = calendar.timegm(dt.datetime.now().timetuple())

params = {'ticket': 'news',
 'templateId': 'default_it',
 'pool': 'cbox5',
#  '_cv': '20240903144034',
#  '_callback': 'jQuery33101711803272944905_1725349376637',
 'lang': 'ko',
 'country': 'KR',
 'objectId': 'news092,0002300844',
 'categoryId': '',
 'pageSize': '20',
 'indexSize': '10',
 'groupId': '',
 'listType': 'OBJECT',
 'pageType': 'more',
 'page': '1',
 'initialize': 'true',
 'followSize': '5',
 'userType': '',
 'useAltSort': 'true',
 'replyPageSize': '20',
 'sort': 'favorite',
 'includeAllStatus': 'true',
 '_': p_time}

response = requests.get(rq_url, headers=headers, params=params)
soup = BeautifulSoup(response.text, 'html.parser')
res_text = response.text

x = res_text.split('(', 1)
js = json.loads(x[1][:-2])
js['result']['morePage']


{'prev': '100000m00000o062lj3we6iz2i',
 'next': '0zik0zh000004062lj6flp1y5p',
 'start': '100000m00000o062lj3we6iz2i',
 'end': '0zik0zc000008062lfxurwlpi4'}

In [52]:
def parse_string_to_dict(input_string):
    # 줄 단위로 분리하여 리스트로 변환
    lines = input_string.strip().split('\n')
    
    # 딕셔너리 생성
    result_dict = {}
    
    # 각 줄을 순회하며 key-value 쌍으로 분리
    for line in lines:
        key, value = line.split(':', 1)  # ':'를 기준으로 split, 최대 1회 split
        result_dict[key.strip()] = value.strip()  # key, value 공백 제거 후 딕셔너리에 추가
    
    return result_dict

In [53]:
input_str = """

ticket: news
templateId: default_it
pool: cbox5
_cv: 20240903144034
_callback: jQuery33101711803272944905_1725349376637
lang: ko
country: KR
objectId: news092,0002300844
categoryId: 
pageSize: 20
indexSize: 10
groupId: 
listType: OBJECT
pageType: more
page: 1
initialize: true
followSize: 5
userType: 
useAltSort: true
replyPageSize: 20
sort: favorite
includeAllStatus: true
_: 1725349376639

"""

parse_string_to_dict(input_str)

{'ticket': 'news',
 'templateId': 'default_it',
 'pool': 'cbox5',
 '_cv': '20240903144034',
 '_callback': 'jQuery33101711803272944905_1725349376637',
 'lang': 'ko',
 'country': 'KR',
 'objectId': 'news092,0002300844',
 'categoryId': '',
 'pageSize': '20',
 'indexSize': '10',
 'groupId': '',
 'listType': 'OBJECT',
 'pageType': 'more',
 'page': '1',
 'initialize': 'true',
 'followSize': '5',
 'userType': '',
 'useAltSort': 'true',
 'replyPageSize': '20',
 'sort': 'favorite',
 'includeAllStatus': 'true',
 '_': '1725349376639'}